In [5]:
# !pip install tokenizers

In [45]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram

tokenizer = Tokenizer(Unigram())
tokenizer

In [46]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output.tokens)

["Hello, y'all! How are you 😁 ?"]


In [47]:
import os

os.listdir('training')

['acq',
 'alum',
 'barley',
 'bop',
 'carcass',
 'castor-oil',
 'cocoa',
 'coconut',
 'coconut-oil',
 'coffee',
 'copper',
 'copra-cake',
 'corn',
 'cotton',
 'cotton-oil',
 'cpi',
 'cpu',
 'crude',
 'dfl',
 'dlr',
 'dmk',
 'earn',
 'fuel',
 'gas',
 'gnp',
 'gold',
 'grain',
 'groundnut',
 'groundnut-oil',
 'heat',
 'hog',
 'housing',
 'income',
 'instal-debt',
 'interest',
 'ipi',
 'iron-steel',
 'jet',
 'jobs',
 'l-cattle',
 'lead',
 'lei',
 'lin-oil',
 'livestock',
 'lumber',
 'meal-feed',
 'money-fx',
 'money-supply',
 'naphtha',
 'nat-gas',
 'nickel',
 'nkr',
 'nzdlr',
 'oat',
 'oilseed',
 'orange',
 'palladium',
 'palm-oil',
 'palmkernel',
 'pet-chem',
 'platinum',
 'potato',
 'propane',
 'rand',
 'rape-oil',
 'rapeseed',
 'reserves',
 'retail',
 'rice',
 'rubber',
 'rye',
 'ship',
 'silver',
 'sorghum',
 'soy-meal',
 'soy-oil',
 'soybean',
 'strategic-metal',
 'sugar',
 'sun-meal',
 'sun-oil',
 'sunseed',
 'tea',
 'tin',
 'trade',
 'unknown',
 'veg-oil',
 'wheat',
 'wpi',
 'yen'

### Radmonly select 10 docs for each class, to build a training set for tokenizer

In [48]:
import re

def process_file_content(content: str) -> str:
    content = re.sub(' +', ' ', content)
    content = content.replace('\n', '')
    return content
    

In [49]:
def load_files_texts(base_relative_path: str, files: list[str]) -> list[str]:
    docs_content = []
    for file in files:
        with open(f"{base_relative_path}/{file}", 'r') as f:
            content = process_file_content(f.read())
            
            docs_content.append(content)
            
    return docs_content
    

In [50]:
from typing import Tuple

def split_into_twe_sets(files: list, splitting_rate: float = 0.9) -> Tuple[list, list]:
    # selected_files = random.sample(files, int(len(files) * splitting_rate))
    index = int(len(files) * splitting_rate)
    return files[:index], files[index:]

    

In [51]:
import random

learning_percent = 0.9
training_docs = []
testing_docs = []

# TODO: refoactor to generators
for folder in os.listdir('training'):
    files = os.listdir(f"training/{folder}")
    files = list(map(lambda x: f"training/{folder}/{x}", files))

    training, testing = split_into_twe_sets(files, learning_percent)

    training_docs.extend(training)
    testing_docs.extend(testing)

print(len(training_docs))

10227


In [52]:
from tokenizers import Tokenizer, normalizers, AddedToken, pre_tokenizers
from tokenizers.normalizers import NFD, StripAccents, Lowercase, Strip, Replace
from tokenizers.pre_tokenizers import Whitespace, Split, Punctuation, Digits
from tokenizers.trainers import UnigramTrainer

In [95]:
digits_replacer = Replace("\d{1,}", "")
new_lines_replacer = Replace("\n", " ")

pre_tokenizer = pre_tokenizers.Sequence([
    Whitespace(),
    Punctuation(),
    Digits()
])

normalizer = normalizers.Sequence([
    digits_replacer,
    NFD(), 
    StripAccents(), 
    Lowercase(), 
    Strip(), 
    new_lines_replacer,
])

special_tokens = [
    "[PAD]",
    "[UNK]",
    "[CLS]",
    "[SEP]",
    "[MASK]",
]

trainer = UnigramTrainer(special_tokens=special_tokens, show_progress=True)

tokenizer.normalizer = normalizer
tokenizer.pre_tokenizer = pre_tokenizer
print(training_docs[0])

training/acq/0000005


In [96]:
tokenizer.train(files=training_docs, trainer=trainer)

In [101]:
tokenizer.save("unigram_tokenizer.json")

In [98]:
tokenizer.encode("""

in the beginning of my working process

""").tokens

['in', 'the', 'beginn', 'ing', 'of', 'm', 'y', 'work', 'ing', 'process']

In [99]:
help(tokenizer.train)

Help on built-in function train:

train(self, files, trainer=None) method of tokenizers.Tokenizer instance
    Train the Tokenizer using the given files.
    
    Reads the files line by line, while keeping all the whitespace, even new lines.
    If you want to train from data store in-memory, you can check
    :meth:`~tokenizers.Tokenizer.train_from_iterator`
    
    Args:
        files (:obj:`List[str]`):
            A list of path to the files that we should use for training
    
        trainer (:obj:`~tokenizers.trainers.Trainer`, `optional`):
            An optional trainer that should be used to train our Model



### Tokenize test docs
#### Pretty good results

In [100]:
for doc in testing_docs[:10]:
    with open(doc) as file:
        print(tokenizer.encode(file.read()).tokens)

['horn', '/', 'har', 'dart', '<', 'h', 'o', 'r', '>', 'unit', 'dispose', 's', '33', 'restaurant', 's', 'las', 've', 'gas', ',', 'n', 'e', 'v', '.', ',', 'apri', 'l', '1', '-', 'horn', 'and', 'har', 'dart', 'co', 'said', 'it', 'dispose', 'd', 'of', '33', 'b', 'o', 'j', 'angle', 's', "'", 'chicken', "'", 'n', 'bisc', 'u', 'its', 'restaurant', 's', 'in', 'flori', 'd', 'a', 'a', 's', 'part', 'of', 'its', 'previous', 'ly', 'announce', 'd', 're', 't', 'rench', 'ment', 'of', 'its', 'b', 'o', 'j', 'angle', 's', "'", 'subsidiar', 'y', '.', 'term', 's', 'were', 'not', 'disclose', 'd', '.', 'the', 'company', 'said', 'the', 'restaurant', 's', 'in', 'o', 'rland', 'o', ',', 't', 'ampa', 'and', 'jackson', 'ville', 'market', 's', 'have', 'incurr', 'ed', 'the', 'major', 'ity', 'of', 'b', 'o', 'j', 'angle', 's', "'", 'operat', 'ing', 'losse', 's', 'over', 'the', 'last', 'two', 'year', 's', '.', 'due', 'to', 'the', 'transaction', ',', 'horn', 'and', 'har', 'dart', 'i', 's', 'project', 'ing', 'a', 'positi

In [58]:
import random

In [59]:
random.sample([5,1,2,3,4], k=3)

[1, 2, 5]

In [60]:
with open(f"training/acq/0000005", 'r') as file:
    print(file.read())



COMPUTER TERMINAL SYSTEMS <CPML> COMPLETES SALE

     COMMACK, N.Y., Feb 26 - Computer Terminal Systems Inc said
it has completed the sale of 200,000 shares of its common
stock, and warrants to acquire an additional one mln shares, to
<Sedio N.V.> of Lugano, Switzerland for 50,000 dlrs.
    The company said the warrants are exercisable for five
years at a purchase price of .125 dlrs per share.
    Computer Terminal said Sedio also has the right to buy
additional shares and increase its total holdings up to 40 pct
of the Computer Terminal's outstanding common stock under
certain circumstances involving change of control at the
company.
    The company said if the conditions occur the warrants would
be exercisable at a price equal to 75 pct of its common stock's
market price at the time, not to exceed 1.50 dlrs per share.
    Computer Terminal also said it sold the technolgy rights to
its Dot Matrix impact technology, including any future
improvements, to <Woodco Inc> of Houston, Tex. 

In [61]:
x = """

INVESTMENT FIRMS CUT CYCLOPS <CYL> STAKE

    WASHINGTON, Feb 26 - A group of affiliated New York
investment firms said they lowered their stake in Cyclops Corp
to 260,500 shares, or 6.4 pct of the total outstanding common
stock, from 370,500 shares, or 9.2 pct.
    In a filing with the Securities and Exchange Commission,
the group, led by Mutual Shares Corp, said it sold 110,000
Cyclops common shares on Feb 17 and 19 for 10.0 mln dlrs.

"""

In [62]:
re.sub("\d{1,}", "", x)

'\n\nINVESTMENT FIRMS CUT CYCLOPS <CYL> STAKE\n\n    WASHINGTON, Feb  - A group of affiliated New York\ninvestment firms said they lowered their stake in Cyclops Corp\nto , shares, or . pct of the total outstanding common\nstock, from , shares, or . pct.\n    In a filing with the Securities and Exchange Commission,\nthe group, led by Mutual Shares Corp, said it sold ,\nCyclops common shares on Feb  and  for . mln dlrs.\n\n'

In [63]:
y = '\n\nINVESTMENT FIRMS CUT CYCLOPS <CYL> STAKE\n\n    WASHINGTON, Feb 26 - A group of affiliated New York\ninvestment firms said they lowered their stake in Cyclops Corp\ntoshares, orpct of the total outstanding common\nstock, fromshares, orpct.\n    In a filing with the Securities and Exchange Commission,\nthe group, led by Mutual Shares Corp, said it sold 110,000\nCyclops common shares on Feb 17 and 19 formln dlrs.\n\n'
re.sub('\n', ' ', y)

'  INVESTMENT FIRMS CUT CYCLOPS <CYL> STAKE      WASHINGTON, Feb 26 - A group of affiliated New York investment firms said they lowered their stake in Cyclops Corp toshares, orpct of the total outstanding common stock, fromshares, orpct.     In a filing with the Securities and Exchange Commission, the group, led by Mutual Shares Corp, said it sold 110,000 Cyclops common shares on Feb 17 and 19 formln dlrs.  '

In [64]:
help(pre_tokenizer.pre_tokenize)

Help on built-in function pre_tokenize:

pre_tokenize(self, pretok) method of tokenizers.pre_tokenizers.Sequence instance
    Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place
    
    This method allows to modify a :class:`~tokenizers.PreTokenizedString` to
    keep track of the pre-tokenization, and leverage the capabilities of the
    :class:`~tokenizers.PreTokenizedString`. If you just want to see the result of
    the pre-tokenization of a raw string, you can use
    :meth:`~tokenizers.pre_tokenizers.PreTokenizer.pre_tokenize_str`
    
    Args:
        pretok (:class:`~tokenizers.PreTokenizedString):
            The pre-tokenized string on which to apply this
            :class:`~tokenizers.pre_tokenizers.PreTokenizer`



In [65]:
# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
 
ps = PorterStemmer()
 
# choose some words to be stemmed
words = ["program", "beginning", "programmer", "programming", "programmers"]
 
for w in words:
    print(w, " : ", ps.stem(w))

program  :  program
beginning  :  begin
programmer  :  programm
programming  :  program
programmers  :  programm
